## install libraries which never used before

In [ ]:
#!pip install joblib
#!pip install klib
#!pip install dtale
#!pip install pandas_profiling

In [1]:
# importing libraries
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read data
df_train= pd.read_csv("e:MallTrain.csv")
df_test= pd.read_csv("e:MallTest.csv")

In [3]:
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df_test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [5]:
df_train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [6]:
df_test.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

### We have two ways to handle null values:

### Either we drop the rows which have null values or replace null values using mean, median, mode, imputer.

### since we have total 8k rows, if we drop 2410 rows then our data will be insufficient to train our model. So, we replace null values.

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 566.0+ KB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Identifier          5681 non-null   object 
 7   Outlet_Establishment_Year  5681 non-null   int64  
 8   Outlet_Size                4075 non-null   object 
 9   Outlet_Location_Type       5681 non-null   object 
 10  Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 332.9+ KB


### Here, we can see "Item_Weight" and "Outlet_Size" contains null values. Since "Item_Weight" is of float type and "Outlet_Size" is of object type. So, we fill mean imputation in "Item_Weight" and mode imputation in "Outlet_Size". 

In [9]:
df_train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


Here, we get mean of "Item_weight" in df_train data is 12.857645.

In [10]:
df_test.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,4705.000000,5681.000000,5681.000000,5681.000000
mean,12.695633,0.065684,141.023273,1997.828903
std,4.664849,0.051252,61.809091,8.372256
min,4.555000,0.000000,31.990000,1985.000000
25%,8.645000,0.027047,94.412000,1987.000000
50%,12.500000,0.054154,141.415400,1999.000000
75%,16.700000,0.093463,186.026600,2004.000000
max,21.350000,0.323637,266.588400,2009.000000


Here, We get mean of Item_Weight in df_test data is 12.695633 .

In [11]:
# here we fill null values in item_weight with its mean
df_train["Item_Weight"].fillna(df_train["Item_Weight"].mean(), inplace= True)
df_test["Item_Weight"].fillna(df_test["Item_Weight"].mean(), inplace= True)

### Now we can see no null values exist in "Item_weight"  Column

In [12]:
df_train.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [13]:
df_test.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

### Now, "Outlet_Size" is categorical column we use mode imputation to fill its null values

In [14]:
df_train["Outlet_Size"].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [15]:
print(df_train["Outlet_Size"].mode())
print(df_test["Outlet_Size"].mode())

0    Medium
dtype: object
0    Medium
dtype: object


In [16]:
df_train["Outlet_Size"].fillna(df_train["Outlet_Size"].mode()[0], inplace= True)
df_test["Outlet_Size"].fillna(df_test["Outlet_Size"].mode()[0], inplace= True)

### Now, no null values exist in "Outlet_Size" column

In [17]:
df_train.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [18]:
df_test.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [19]:
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Selecting features based on general requirements

As we can see two column in given data has no meaning. So, we can drop those two columns from "df_train" and "df_test".

Columns are "Item_Identifier" and Outlet_Identifier"

In [20]:
df_train.drop(['Item_Identifier', 'Outlet_Identifier'],axis=1,inplace=True)

In [21]:
df_test.drop(['Item_Identifier', 'Outlet_Identifier'],axis=1,inplace=True)

now we can see there is no column present named as Item_Identifier and Outlet_Identifier in df_train and df_test

In [22]:
df_train.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,1998,Medium,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052


In [23]:
df_test.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,20.750000,Low Fat,0.007565,Snack Foods,107.8622,1999,Medium,Tier 1,Supermarket Type1
1,8.300000,reg,0.038428,Dairy,87.3198,2007,Medium,Tier 2,Supermarket Type1
2,14.600000,Low Fat,0.099575,Others,241.7538,1998,Medium,Tier 3,Grocery Store
3,7.315000,Low Fat,0.015388,Snack Foods,155.0340,2007,Medium,Tier 2,Supermarket Type1
4,12.695633,Regular,0.118599,Dairy,234.2300,1985,Medium,Tier 3,Supermarket Type3


# EDA (Exploratory data analysis)

Exploratory Data Analysis an approach for analyzing datasets to summarize their main characteristics, often with visual methods

## EDA using dtale Library

In [24]:
import dtale

In [25]:
dtale.show(df_train)

## EDA using pandas profiling

In [26]:
from pandas_profiling import ProfileReport

In [27]:
profile = ProfileReport(df_train, title= "Pandas Profiling Report")

In [28]:
profile

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
plt.figure(figsize= (10, 5))
sns.heatmap(df_train.corr(), annot= True)
plt.show()
# use annot to put values in boxes

D:\Users\AGRAHARI\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning:

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



## EDA using klib library

Klib library is not much useful haere becouse we already  

* cleaned our data    

* deal with missing values   

* removed duplicate values  

In [30]:
import klib

In [31]:
# klib.describe- function for visualizing datasets
klib.cat_plot(df_train) # return the visualization of the number and frequency of categorical features

GridSpec(6, 5)

In [32]:
klib.corr_mat(df_train)  # returns a colour encoded matrix

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_Weight,1.00,-0.01,0.02,-0.01,0.01
Item_Visibility,-0.01,1.00,-0.00,-0.07,-0.13
Item_MRP,0.02,-0.00,1.00,0.01,0.57
Outlet_Establishment_Year,-0.01,-0.07,0.01,1.00,-0.05
Item_Outlet_Sales,0.01,-0.13,0.57,-0.05,1.00


In [33]:
klib.corr_plot(df_train)

<AxesSubplot:title={'center':'Feature-correlation (pearson)'}>

In [34]:
klib.dist_plot(df_train)    # Returns a distribution plot for every numeric feature 

<AxesSubplot:xlabel='Item_Outlet_Sales', ylabel='Density'>

In [35]:
klib.missingval_plot(df_train) # Returns a figure containing information about missing values

No missing values found in the dataset.


## Data Cleaning using klib

In [36]:
# klib.clean- function for cleaning dataset
klib.data_cleaning(df_train)   # Performs data cleaning drop duplicate, empty rows/column, adjust datatypes.....


# reduced memory by 0.27MB by replacing capital letters with small letters in column name

Shape of cleaned data: (8523, 10)Remaining NAs: 0

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 0.27 MB (-55.1%)



,item_weight,item_fat_content,item_visibility,item_type,item_mrp,outlet_establishment_year,outlet_size,outlet_location_type,outlet_type,item_outlet_sales
0,9.3,Low Fat,0.016047,Dairy,249.809204,1999,Medium,Tier 1,Supermarket Type1,3735.137939
1,5.92,Regular,0.019278,Soft Drinks,48.269199,2009,Medium,Tier 3,Supermarket Type2,443.422791
2,17.5,Low Fat,0.01676,Meat,141.617996,1999,Medium,Tier 1,Supermarket Type1,2097.27002
3,19.200001,Regular,0.0,Fruits and Vegetables,182.095001,1998,Medium,Tier 3,Grocery Store,732.380005
4,8.93,Low Fat,0.0,Household,53.861401,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,Snack Foods,214.521805,1987,High,Tier 3,Supermarket Type1,2778.383301
8519,8.38,Regular,0.046982,Baking Goods,108.156998,2002,Medium,Tier 2,Supermarket Type1,549.284973
8520,10.6,Low Fat,0.035186,Health and Hygiene,85.122398,2004,Small,Tier 2,Supermarket Type1,1193.113647
8521,7.21,Regular,0.145221,Snack Foods,103.133202,2009,Medium,Tier 3,Supermarket Type2,1845.597656


In [37]:
klib.clean_column_names(df_train)   # clean and standardise column names, also called inside data_cleaning()

,item_weight,item_fat_content,item_visibility,item_type,item_mrp,outlet_establishment_year,outlet_size,outlet_location_type,outlet_type,item_outlet_sales
0,9.300,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.920,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.500,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,1998,Medium,Tier 3,Grocery Store,732.3800
4,8.930,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,Snack Foods,214.5218,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,8.380,Regular,0.046982,Baking Goods,108.1570,2002,Medium,Tier 2,Supermarket Type1,549.2850
8520,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,7.210,Regular,0.145221,Snack Foods,103.1332,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [38]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   item_weight                8523 non-null   float64
 1   item_fat_content           8523 non-null   object 
 2   item_visibility            8523 non-null   float64
 3   item_type                  8523 non-null   object 
 4   item_mrp                   8523 non-null   float64
 5   outlet_establishment_year  8523 non-null   int64  
 6   outlet_size                8523 non-null   object 
 7   outlet_location_type       8523 non-null   object 
 8   outlet_type                8523 non-null   object 
 9   item_outlet_sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 499.5+ KB


In [39]:
klib.convert_datatypes(df_train)   # Convert existion to more efficient dtypes also called inside data_cleaning()

,item_weight,item_fat_content,item_visibility,item_type,item_mrp,outlet_establishment_year,outlet_size,outlet_location_type,outlet_type,item_outlet_sales
0,9.3,Low Fat,0.016047,Dairy,249.809204,1999,Medium,Tier 1,Supermarket Type1,3735.137939
1,5.92,Regular,0.019278,Soft Drinks,48.269199,2009,Medium,Tier 3,Supermarket Type2,443.422791
2,17.5,Low Fat,0.01676,Meat,141.617996,1999,Medium,Tier 1,Supermarket Type1,2097.27002
3,19.200001,Regular,0.0,Fruits and Vegetables,182.095001,1998,Medium,Tier 3,Grocery Store,732.380005
4,8.93,Low Fat,0.0,Household,53.861401,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,Snack Foods,214.521805,1987,High,Tier 3,Supermarket Type1,2778.383301
8519,8.38,Regular,0.046982,Baking Goods,108.156998,2002,Medium,Tier 2,Supermarket Type1,549.284973
8520,10.6,Low Fat,0.035186,Health and Hygiene,85.122398,2004,Small,Tier 2,Supermarket Type1,1193.113647
8521,7.21,Regular,0.145221,Snack Foods,103.133202,2009,Medium,Tier 3,Supermarket Type2,1845.597656


In [40]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   item_weight                8523 non-null   float64
 1   item_fat_content           8523 non-null   object 
 2   item_visibility            8523 non-null   float64
 3   item_type                  8523 non-null   object 
 4   item_mrp                   8523 non-null   float64
 5   outlet_establishment_year  8523 non-null   int64  
 6   outlet_size                8523 non-null   object 
 7   outlet_location_type       8523 non-null   object 
 8   outlet_type                8523 non-null   object 
 9   item_outlet_sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 499.5+ KB


In [41]:
klib.mv_col_handling(df_train)   # Drop column having null values 

# No column with missing values exist here

,item_weight,item_fat_content,item_visibility,item_type,item_mrp,outlet_establishment_year,outlet_size,outlet_location_type,outlet_type,item_outlet_sales
0,9.300,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.920,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.500,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,1998,Medium,Tier 3,Grocery Store,732.3800
4,8.930,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,Snack Foods,214.5218,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,8.380,Regular,0.046982,Baking Goods,108.1570,2002,Medium,Tier 2,Supermarket Type1,549.2850
8520,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,7.210,Regular,0.145221,Snack Foods,103.1332,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [42]:
klib.pool_duplicate_subsets(df_train)    # Pools subsets of columns based on duplicate with min. loss of information

,item_visibility,item_mrp,item_outlet_sales,pooled_vars
0,0.016047,249.8092,3735.1380,0
1,0.019278,48.2692,443.4228,1
2,0.016760,141.6180,2097.2700,2
3,0.000000,182.0950,732.3800,3
4,0.000000,53.8614,994.7052,4
...,...,...,...,...
8518,0.056783,214.5218,2778.3834,8518
8519,0.046982,108.1570,549.2850,8519
8520,0.035186,85.1224,1193.1136,8520
8521,0.145221,103.1332,1845.5976,8521


## Preprocessing Task Before Model Building
* Normalization
* Partitioning

### 1. Label Encoding :
        Transform categorical values into Numerical values.

we can perform "OneHotEncodig" also but here we are doing "LabelEncoding"

In [43]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

In [44]:
df_train= df_train.apply(le.fit_transform)

In [45]:
df_train

,item_weight,item_fat_content,item_visibility,item_type,item_mrp,outlet_establishment_year,outlet_size,outlet_location_type,outlet_type,item_outlet_sales
0,284,1,664,4,5592,4,1,0,1,2540
1,57,2,880,14,473,8,1,2,2,422
2,376,1,715,10,2901,4,1,0,1,1639
3,393,2,0,6,4227,3,1,2,0,670
4,265,1,0,9,627,1,0,2,1,865
...,...,...,...,...,...,...,...,...,...,...
8518,125,1,3912,13,4955,1,0,2,1,2047
8519,233,2,3278,0,2023,5,1,1,1,516
8520,299,1,2302,8,1263,6,2,1,1,1018
8521,149,2,7175,13,1857,8,1,2,2,1466


### 2. OneHotEncoding

In [46]:
# we can perform OneHotEncoding using pandas also
df_train= pd.get_dummies(df_train, columns= ["item_fat_content", "outlet_size", "outlet_location_type", "outlet_type"])

In [47]:
df_train

,item_weight,item_visibility,item_type,item_mrp,outlet_establishment_year,item_outlet_sales,item_fat_content_0,item_fat_content_1,item_fat_content_2,item_fat_content_3,...,outlet_size_0,outlet_size_1,outlet_size_2,outlet_location_type_0,outlet_location_type_1,outlet_location_type_2,outlet_type_0,outlet_type_1,outlet_type_2,outlet_type_3
0,284,664,4,5592,4,2540,0,1,0,0,...,0,1,0,1,0,0,0,1,0,0
1,57,880,14,473,8,422,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0
2,376,715,10,2901,4,1639,0,1,0,0,...,0,1,0,1,0,0,0,1,0,0
3,393,0,6,4227,3,670,0,0,1,0,...,0,1,0,0,0,1,1,0,0,0
4,265,0,9,627,1,865,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,125,3912,13,4955,1,2047,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0
8519,233,3278,0,2023,5,516,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
8520,299,2302,8,1263,6,1018,0,1,0,0,...,0,0,1,0,1,0,0,1,0,0
8521,149,7175,13,1857,8,1466,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0


## 3. Splitting our Data into training and test Data

In [48]:
X= df_train.drop("item_outlet_sales", axis= 1)
Y= df_train["item_outlet_sales"]

In [49]:
Y

0       2540
1        422
2       1639
3        670
4        865
        ... 
8518    2047
8519     516
8520    1018
8521    1466
8522     697
Name: item_outlet_sales, Length: 8523, dtype: int32

In [50]:
X

,item_weight,item_visibility,item_type,item_mrp,outlet_establishment_year,item_fat_content_0,item_fat_content_1,item_fat_content_2,item_fat_content_3,item_fat_content_4,outlet_size_0,outlet_size_1,outlet_size_2,outlet_location_type_0,outlet_location_type_1,outlet_location_type_2,outlet_type_0,outlet_type_1,outlet_type_2,outlet_type_3
0,284,664,4,5592,4,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0
1,57,880,14,473,8,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
2,376,715,10,2901,4,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0
3,393,0,6,4227,3,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0
4,265,0,9,627,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,125,3912,13,4955,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0
8519,233,3278,0,2023,5,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
8520,299,2302,8,1263,6,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0
8521,149,7175,13,1857,8,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, random_state= 101, test_size= 0.2)

## 4. Standardization
    scaling technique where the values are centered around the mean with a unit standard deviation.
    
    This means that the mean of the attribute becomes zero and the resultant distribution has a unit standard deviation

In [52]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

In [53]:
X_train_std= sc.fit_transform(X_train)
X_test_std= sc.transform(X_test)

In [54]:
X_train_std

array([[ 1.00439018,  0.96533577, -0.75847359, ...,  0.73160085,
        -0.34892775, -0.35759513],
       [-1.23760194,  0.19186569,  1.60759199, ...,  0.73160085,
        -0.34892775, -0.35759513],
       [ 1.01496562,  0.29157407,  0.66116576, ...,  0.73160085,
        -0.34892775, -0.35759513],
       ...,
       [ 0.23238346, -1.19862827,  0.18795264, ...,  0.73160085,
        -0.34892775, -0.35759513],
       [-0.09545501,  1.29992208, -1.70489982, ...,  0.73160085,
        -0.34892775, -0.35759513],
       [ 0.62367454,  1.53980627,  0.4245592 , ..., -1.36686554,
         2.8659228 , -0.35759513]])

In [55]:
X_test_std

array([[ 0.06317651,  0.04042575,  0.18795264, ..., -1.36686554,
         2.8659228 , -0.35759513],
       [ 0.86690953, -0.4451833 ,  1.37098543, ...,  0.73160085,
        -0.34892775, -0.35759513],
       [-1.15299846,  0.48515019, -1.46829326, ...,  0.73160085,
        -0.34892775, -0.35759513],
       ...,
       [ 0.61309911,  1.08506928,  0.89777232, ..., -1.36686554,
        -0.34892775, -0.35759513],
       [ 0.77173062, -1.55115374,  0.4245592 , ..., -1.36686554,
         2.8659228 , -0.35759513],
       [-2.33744713, -1.47230485,  0.66116576, ...,  0.73160085,
        -0.34892775, -0.35759513]])

In [56]:
Y_train

3684     144
1935    1311
5142    1242
4978    1426
2299    2462
        ... 
599     3137
5695    1193
8006    1684
1361    2001
1547     746
Name: item_outlet_sales, Length: 6818, dtype: int32

In [57]:
Y_test

8179     801
8355    2056
3411    1529
7089     779
6954    1858
        ... 
1317    1386
4996     808
531      356
3891    1144
6629    1839
Name: item_outlet_sales, Length: 1705, dtype: int32

## Model Building

In [58]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression()

In [59]:
lr.fit(X_train_std, Y_train)

LinearRegression()

In [60]:
X_test

,item_weight,item_visibility,item_type,item_mrp,outlet_establishment_year,item_fat_content_0,item_fat_content_1,item_fat_content_2,item_fat_content_3,item_fat_content_4,outlet_size_0,outlet_size_1,outlet_size_2,outlet_location_type_0,outlet_location_type_1,outlet_location_type_2,outlet_type_0,outlet_type_1,outlet_type_2,outlet_type_3
8179,305,3815,8,1753,8,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0
8355,381,2651,13,3152,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0
3411,190,4881,1,1062,2,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0
7089,408,3427,6,207,7,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0
6954,175,1624,3,3291,5,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,369,6144,13,3381,4,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0
4996,241,1606,14,3308,2,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0
531,357,6319,11,4321,3,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0
3891,372,0,9,2252,8,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0


In [61]:
Y_pred_lr= lr.predict(X_test_std)
Y_pred_lr

array([1075.12863866, 1672.50363866, 1077.50363866, ...,  717.75363866,
       1241.12863866, 1687.75363866])

In [62]:
Y_test

8179     801
8355    2056
3411    1529
7089     779
6954    1858
        ... 
1317    1386
4996     808
531      356
3891    1144
6629    1839
Name: item_outlet_sales, Length: 1705, dtype: int32

In [63]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [64]:
print(r2_score(Y_test, Y_pred_lr))
print(mean_absolute_error(Y_test, Y_pred_lr))
print(np.sqrt(mean_squared_error(Y_test, Y_pred_lr)))

0.6236648387827346
446.2716716284539
558.2554804265103


In [65]:
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()

In [82]:
rf.fit(X_train_std, Y_train)

RandomForestRegressor()

In [83]:
Y_pred_rf= rf.predict(X_test_std)
Y_pred_rf

array([1413.77, 2257.19, 1070.68, ...,  381.55, 1320.04, 1861.92])

In [84]:
Y_test

8179     801
8355    2056
3411    1529
7089     779
6954    1858
        ... 
1317    1386
4996     808
531      356
3891    1144
6629    1839
Name: item_outlet_sales, Length: 1705, dtype: int32

In [85]:
print(r2_score(Y_test, Y_pred_rf))
print(mean_absolute_error(Y_test, Y_pred_rf))
print(np.sqrt(mean_squared_error(Y_test, Y_pred_rf)))

0.6119157695104241
430.7809912023461
566.9027906823712


## Hyper Parameter Tuning

In [75]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

# Define model and parameters
model= RandomForestRegressor()
n_estimators= [10, 100, 1000]
max_depth= range(1, 31)
min_samples_leaf= np.linspace(0.1, 1.0)
max_features= ["auto", "sqrt", "log2"]
min_samples_split= np.linspace(0.1, 1.0, 10)

# Define GridSearch
grid= dict(n_estimators= n_estimators)
cv= RepeatedStratifiedKFold(n_splits= 5, n_repeats= 3, random_state= 101)
grid_search_forest= GridSearchCV(estimator= model, param_grid= grid, n_jobs= -1,
                                 scoring= 'r2', error_score= 0, verbose= 2, cv= 2)
grid_search_forest.fit(X_train_std, Y_train)

# Summarise results
print(f"Best: {grid_search_forest.best_score_:.3f} using {grid_search_forest.best_params_}")
means= grid_search_forest.cv_results_["mean_test_score"]
stds= grid_search_forest.cv_results_["std_test_score"]
params= grid_search_forest.cv_results_["params"]

for mean, stdev, param in zip(means, stds, params):
    print(f"{mean:.3f} ({stdev:.3f}) with: {param}")
          

Fitting 2 folds for each of 3 candidates, totalling 6 fits
Best: 0.623 using {'n_estimators': 1000}
0.593 (0.009) with: {'n_estimators': 10}
0.620 (0.004) with: {'n_estimators': 100}
0.623 (0.003) with: {'n_estimators': 1000}


In [76]:
grid_search_forest.best_params_

{'n_estimators': 1000}

In [77]:
grid_search_forest.best_score_

0.6226115362827924

In [81]:
Y_pred_rf_grid= grid_search_forest.predict(X_test_std)
Y_pred_rf_grid

array([1348.842, 2387.108, 1091.941, ...,  347.447, 1300.816, 1771.586])

In [86]:
r2_score(Y_test, Y_pred_rf_grid)

0.6146499956932014

# Save Your model

In [87]:
import joblib

In [89]:
joblib.dump(grid_search_forest,r"E:\model\random_forest_grid.sav")

['E:\\model\\random_forest_grid.sav']

In [91]:
model= joblib.load(r"E:\model\random_forest_grid.sav")

MemoryError: Unable to allocate 395. KiB for an array with shape (8417,) and data type {'names':['left_child','right_child','feature','threshold','impurity','n_node_samples','weighted_n_node_samples'], 'formats':['<i4','<i4','<i4','<f8','<f8','<i4','<f8'], 'offsets':[0,4,8,16,24,32,40], 'itemsize':48}

In [ ]:
model.predict(x_test)